# ⚙️📜🗣️  Vizualizace přirozeného jazyka

V tomto cvičení budeme vizualizovat data týkající se fenoménu [řetězových e-mailů](https://cs.wikipedia.org/wiki/%C5%98et%C4%9Bzov%C3%BD_e-mail), které kromě zpráv od [Nigérijských princů](https://cs.wikipedia.org/wiki/Nigerijsk%C3%A9_dopisy), [phishingových mailů](https://cs.wikipedia.org/wiki/Phishing), [ocbchodních sdělení](https://cs.wikipedia.org/wiki/Phishing) a dalších druhů e-mailů mohou sídlit v našich inboxech. Řetězové e-maily fungují na principu přeposílání zprávy, která často obsahuje propagandu, dezinformace, hoaxy a nebo poplašné zprávy. Zprávu chceme zaslat co nejvíce lidem a tím exponenciálně šířit informaci, která má za cíl zasít ve společnosti myšlenku nebo názor, na kterém pak bude autor/skupina autorů nějakým způsobem profitovat.

Vy sami se nejspíše dobře orientujete v tom, jaké e-maily máte číst, na které klikat a kterým věřit, ale bohužel ne všichni mají stejně vycvičenou intuici. Někteří lidé dokonce berou obsah řetězových e-mailů jako relevantní informační zdroj. 

*🤷🏻 "Musí to být přece pravda, když mi to přišlo od někoho, koho znám." 🤷‍♀️*

V poslední době je čím dál tím více upozorňováno na problematiku řetězových e-mailů. Toto cvičení slouží jako taková malá sonda do tohoto světa. Dataset pochází z databáze [řetězových mailů](https://eldariel.cesti-elfove.cz). 

In [ ]:
import pandas as pd
import os
import operator
import unidecode

import plotly

plotly.offline.init_notebook_mode(connected=True)

import numpy as np

from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
df = pd.read_csv("./chain_mails.csv", index_col=0)
df.Datum = pd.to_datetime(df.Datum)
df.drop(index=df[df["Datum"].dt.year < 2018].index, inplace=True)
df["Předmět"] = df["Předmět"].astype(str)

df

🔎 Na portálu jsou ve větším množství e-maily od roku 2019 do současnosti (19. 9. 2022). Náš dataset obsahuje 11 583 záznamů o 5 sloupcích: `ID`, `Předmět`, `Štítky` (seznam témat probíraných v e-mailu), `Zmínění` (lidé a instituce zmínění v e-mailu) a `Datum`.  

# 📧 Jak často e-maily chodí?

Jako první nás může napadnout otázka: "Jak často chodí takové řetězové e-maily?"

In [ ]:
df = df[
        (df["Datum"].dt.year >= 2019)
        | ((df["Datum"].dt.year == 2022) & (df["Datum"].dt.month <= 9))
    ]

fig = (
    df.set_index("Datum")
    .resample("D")
    .size()
    .rename("počet")
    .plot(backend="plotly", title="Denní počty řetězových mailů")
)


fig.show()

🔎 Z grafu vidíme, že řetězové e-maily chodí opravdu každý den, ale počty e-mailů jsou proměnlivé. S velkou pravděpodobností tyto výkyvy souvisí s událostmi a náladami ve společnosti. Zkusme proto přidat pro srovnání pár událostí, které v posledních dvou letech rezonovaly Českem.

In [ ]:
def show_events():
    fig.add_vline(
        x=pd.to_datetime("24/02/2022", format="%d/%m/%Y").timestamp() * 1000,
        line_width=2,
        line_dash="dash",
        line_color="red",
        annotation_text="Ruská invaze na Ukrajinu",
        annotation_textangle=90,
    )
    fig.add_vline(
        x=pd.to_datetime("08/10/2021", format="%d/%m/%Y").timestamp() * 1000,
        line_width=2,
        line_dash="dash",
        line_color="yellow",
        annotation_text="Volby do PSP a hospitalizace prezidenta",
        annotation_textangle=90,
    )
    fig.add_vline(
        x=pd.to_datetime("18/04/2021", format="%d/%m/%Y").timestamp() * 1000,
        line_width=2,
        line_dash="dash",
        line_color="green",
        annotation_text="BIS + Vrbětice",
        annotation_textangle=90,
    )
    fig.add_vline(
        x=pd.to_datetime("27/12/2020", format="%d/%m/%Y").timestamp() * 1000,
        line_width=2,
        line_dash="dash",
        line_color="pink",
        annotation_text="Očkování proti COVID-19 v ČR",
        annotation_textangle=90,
    )
    fig.add_vline(
        x=pd.to_datetime("23/09/2020", format="%d/%m/%Y").timestamp() * 1000,
        line_width=2,
        line_dash="dash",
        line_color="cyan",
        annotation_text="Otrava řeky Bečva",
        annotation_textangle=90,
    )
    fig.add_vline(
        x=pd.to_datetime("02/07/2021", format="%d/%m/%Y").timestamp() * 1000,
        line_width=2,
        line_dash="dash",
        line_color="purple",
        annotation_text="USA opouští Afghánistán",
        annotation_textangle=90,
    )
    fig.add_vline(
        x=pd.to_datetime("01/05/2022", format="%d/%m/%Y").timestamp() * 1000,
        line_width=2,
        line_dash="dash",
        line_color="purple",
        annotation_text="Obléhání Mariupolu",
        annotation_textangle=90,
    )
    fig.show()

In [ ]:
show_events()

🔎 Z grafu vidíme, že tyto události opravdu předcházely nárůstu v počtu detekovaných řetězových e-mailů, avšak nemůžeme s jistotou říct, že tyto události byly přímo spouštěčem.

## 🔁 Sezónnost v aktivitě

Mohlo by nás také zajímat, jestli třeba v nějakém období není situace okolo řetězových e-mailů klidnější než jindy a zda-li jsou naopak některé měsíce těmito e-maily nabité.

In [ ]:
def get_weekly_counts(x):
    """Returns a weekly count of e-mails."""
    return x.groupby("Datum").size().resample("w").size().rename("count")

👉 Spočítáme týdenní počty e-mailů pro jednotlivé roky:

In [ ]:
ts = (
    df.assign(year=df["Datum"].dt.year)
    .groupby("year")
    .apply(get_weekly_counts)
    .reset_index()
)

In [ ]:
fig = (
    ts.groupby("year")
    .apply(lambda x: x.assign(week=x.Datum.dt.isocalendar().week % 52))
    .sort_values(by="week")
    .plot(
        backend="plotly",
        x="week",
        y="count",
        color="year",
        title="Sezónnost řetězových e-mailů (týdenní)",
    )
)
fig.add_vline(
    23,
    annotation_text="Červen",
    line_dash="dash",
    line_color="purple",
)
fig.add_vline(
    36,
    annotation_text="Září",
    line_dash="dash",
    line_color="purple",
)
fig.update_xaxes(range=[2, 51])

🔎 Z grafu je vidět, že v letech 2020 a 2021 byli autoři řetězových mailů z počátku roku celkem aktivní, do dubna se počet e-mailů zvětšoval, ale s příchodem letních prázdnin došlo k značnému poklesu. Tento pokles celkem znatelně souvisí se [sezónností v období dovolených](https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Seasonality_in_the_tourist_accommodation_sector).

# 📦 Co je obsahem e-mailů? 

Další dobrá otázka. Problém je, že asi nezvládneme přečíst všech téměř 12 tisíc e-mailů, tím spíš když bychom je museli analyzovat. Naštěstí můžeme tento proces automatizovat pomocí programování, regulárních výrazů a špetkou statistiky. Jelikož předměty e-mailů často slouží ke shrnutí obsahu zprávy, začněme zde.

In [ ]:
from collections import Counter
from functools import reduce
import re

👉 Nejdříve pomocí regulárních výrazů vyextrahujeme jednotlivá slova z předmětů (více viz modul [re](https://docs.python.org/3/howto/regex.html)). 

In [ ]:
tokens = df["Předmět"].str.findall(r"[\w]+", flags=re.IGNORECASE)

In [ ]:
words = reduce(lambda a, b: a + b, tokens.values.tolist())

In [ ]:
# prvních 14 slov
words[:14]

👉 Ve výsledku budeme mít seznam všech slov nacházejících se v předmětech e-mailů. Uděláme si tedy statistiku výskytů a podíváme se na nejčastěji použitá slova.

In [ ]:
subject_word_count = Counter(words)

In [ ]:
# 10 nejčastějších slov
subject_word_count.most_common()[:10]

🔎 Z výpisu ale vidíme, že nejčastější slova jsou spojky a zájmena, včetně zvratných. Je třeba trochu zlepšit naši detekci.

## 🛑 Stop words
Jednou základní operací NLP je odebrání tzv. stop words (spojky, předložky, apod.).

👉 Jednou z možností je pomocí knihovny `requests` si stáhnout seznam stopwords z internetu, např.:

In [ ]:
#import requests
#stop_words = requests.get(
#    "https://raw.githubusercontent.com/Alir3z4/stop-words/master/czech.txt"
#).text.split()

Nebo můžete nahrát vlastní soubor, např. který jsme si pro danou úlohu připravili: 

In [ ]:
with open('stopwords.txt') as e:
  stop_words = e.read()

In [ ]:
# count of stopwords
len(stop_words)

👉 A odfiltrujeme stop slova.

In [ ]:
from functools import lru_cache


@lru_cache(maxsize=None)
def is_stop_word(word):
    return word in stop_words

In [ ]:
subject_word_count = Counter(
    filter(
        lambda x: not is_stop_word(x) and not x.isnumeric() and not len(x) < 2,
        map(lambda x: x.lower(), words),
    )
)

In [ ]:
subject_word_count.most_common()[:10]

🔎 Z výpisu vidíme, že odebrání stop slov opravdu pomohlo a v prvních `10 nejčastějších slovech` se už žádná spojka nenachází. Máme zde ale jiný problém - osmé a desáté nejčastější slovo, je vlastně jedno a to samé, jen má jiný tvar. V obou případech jde o Ukrajinu. Počítač bohužel tohle sám nerozliší, máme ale k dispozici lingvistické nástroje, které ano.

## ✨ Lemmatizace

Jedním takovým nástrojem je lemmatizace textu. Můžeme využít nástroj [majka](https://nlp.fi.muni.cz/ma/) od kolegů z Masarykovy Univerzity v Brně. Díky tomuto nástroji můžeme najít lemmata (základní tvary) jednotlivých slov.

In [ ]:
import majka

In [ ]:
morph = majka.Majka("./majka.w-lt")

In [ ]:
def lemmatize(word):
    """Returns a lemmatized form of a word, or the word if the lemma does not exist."""
    lemma = word
    try:
        lemma = morph.find(word)[0]["lemma"]
    except IndexError:
        lemma = word
    return lemma

👉Například slovo `"Nůžek"` jednoduše převedeme na  

In [ ]:
lemmatize("Nůžek")

In [ ]:
def preprocess(words):
    """Filters out a list of worlds and converts them to lemmas."""
    out = []
    for word in words:
        if is_stop_word(word.lower()) or word.isnumeric() or len(word) < 2:
            continue
        out.append(lemmatize(word).lower())
    return out

In [ ]:
subject_word_count = Counter(preprocess(words))

In [ ]:
subject_word_count.most_common()[:10]

🔎 A je to. Podařilo se nám spojit slova se stejným významem a tím značně zlepšit kvalitu naší statistiky.

## ☁️ Word Cloud
Jedna z forem vizualizace přirozeného jazyka je wordcloud, který kombinuje četnost slov a slova samotná tak, že častější slova mají větší velikost a naopak. Využijeme k tomu balíček [`wordcloud`](https://amueller.github.io/word_cloud/index.html).

In [ ]:
import wordcloud

In [ ]:
cloud = wordcloud.WordCloud(background_color="white", max_font_size=70, width=1000, height=500)
cloud.fit_words(subject_word_count)
# show image in matplotlib way
plt.figure(figsize=(16, 10))
plt.imshow(cloud) #interpolation="bilinear"
plt.axis("off")
plt.show()

Pamatujete si na 3. cvičení, kdy jsme si povídali o rozhraních `matplotlib`u? Podívejte se na kód k wordcloudu a zkuste jej za ⭐️**dobrovolný domácí úkol**⭐️ převést do objektově orientované reprezentace.

Pozn.: U `plt.imshow()` se můžete setkat s parametrem `interpolation`, u wordcloudů nastaveným na hodnotu `bilinear`. Více si o této transformaci můžete přečíst v [📖dokumentaci](https://matplotlib.org/stable/gallery/images_contours_and_fields/interpolation_methods.html).

## 😐 Sentiment

Další věc, kterou můžeme v textu sledovat, je sentiment. Ten se často dělí na _negativní_ 😖, _neutrální_ 😐 a _pozitivní_ 🙂. K tomu použijeme druhou část datasetu, která obsahuje dodatečné informace k jednotlivým e-mailům včetně klasifikace obsahu e-mailu do zmíněných kategorií. 🙊

In [ ]:
meta = pd.read_csv("./chain_mails_meta_info.csv", index_col="id").fillna("")
meta['Datum'] = pd.to_datetime(meta['Datum'])
meta = meta[meta['Datum'].dt.year > 2019]

In [ ]:
meta.tail(5)

In [ ]:
positive_mentions = reduce(
    lambda a, b: a + b,
    meta["positive_mentions"].fillna("").apply(lambda x: x.split(",")),
)

In [ ]:
neutral_mentions = reduce(
    lambda a, b: a + b,
    meta["neutral_mentions"].fillna("").apply(lambda x: x.split(",")),
)

In [ ]:
negative_mentions = reduce(
    lambda a, b: a + b,
    meta["negative_mentions"].fillna("").apply(lambda x: x.split(",")),
)

In [ ]:
positive = Counter(filter(bool, positive_mentions))
neutral = Counter(filter(bool, neutral_mentions))
negative = Counter(filter(bool, negative_mentions))

In [ ]:
positive.most_common()[:10]

In [ ]:
neutral.most_common()[:10]

In [ ]:
negative.most_common()[:10]

🔎 Ze statistik je vidět, že nejvíce pozitivně se mluvilo o Rusku, což podporuje i tvrzení, že řetězové e-maily jsou využívány jako nástroj ruské propagandy. 🫢 Neutrálně se mluvilo o důchodcích, což může reflektovat i to, že [každý pátý důchodce přeposílá řetězové e-maily](https://zpravy.aktualne.cz/domaci/seniori-retezove-emaily-trollove-elpida/r~4bc1f5faef4a11e99d020cc47ab5f122/). Negativně se pak nejvíce mluví o Evropské unii.

### 🥧 Koláčový graf a sentiment

Můžeme se také podívat na poměr pozitivních, negativních a neutrálních zmínění jednotlivých osob v e-mailech.

In [ ]:
intersection = sorted(
    list(set(positive).union(set(negative)).union(set(neutral)))
) # vezmeme sjednocení všech množin

👉 Tady budeme uvažovat jen zmínky, které jsou ve všech třech kategoriích:

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets


def get_pie_chart(subject):
    """Creates a pie chart of sentiment shares."""
    fig = go.Figure(
        data=[
            go.Pie(
                title=subject,
                labels=["Neutral", "Negative", "Positve"],
                values=[neutral.get(subject, 0), negative.get(subject, 0), positive.get(subject, 0)],
                hole=0.4,
                sort=False,
            )
        ]
    )
    fig.update_layout(
    title="Koláčový graf sentimentu k subjektům řetězových e-mailů")
    fig.show()


widgets.interact(get_pie_chart, subject=sorted(intersection));

### 🎨 Obarvení slov ve wordcloudech
Nebo můžeme obarvit jednotlivé zmínky podle převažujícího sentimentu.

In [ ]:
cloud = wordcloud.WordCloud(max_font_size=70, width=1000, height=500)
cloud.fit_words({k: v for k, v in negative.most_common()});
cloud.fit_words({k: v for k, v in positive.most_common()});
cloud.fit_words({k: v for k, v in neutral.most_common()});

In [ ]:
red = "#DD1C1A"
green = "#29BF12"
blue = "#6878DE"

In [ ]:
def color(word, **kwargs):
    """Colors words based on sentiment (negative, positive, neutral)."""
    colors = (red, blue, green)
    neg_count = negative[word] if word in negative else 0
    neu_count = neutral[word] if word in neutral else 0
    pos_count = positive[word] if word in positive else 0
    return colors[np.argmax([neg_count, neu_count, pos_count])]

In [ ]:
color("Miroslav Kalousek") is red

In [ ]:
cloud.recolor(color_func=color);

In [ ]:
plt.figure(figsize=(16, 10))
plt.imshow(cloud) #interpolation="bicubic"
plt.axis("off")
plt.show()

🔎 Teď už máme kompletní přehled o tom, koho nebo co zmiňovaly řetězové e-maily a jaký v daným případech převládá nálada. V e-mailech jsou pozitivně zmiňováni někteří čeští politici, Čína, Donald Trump, Putin nebo dokonce Stalin (diktátor, vzpomeňte). Neutrálně jsou zmiňováni senioři, různé státy jako například Turecko, Bulharsko nebo Česká republika, ale také (nečekaně) ukrajinský prezident Volodymyr Zelenskyj. Negativně se psalo o Václavu Havlovi, USA, Angele Merkelové nebo Ukrajině.

# 💬 Témata řetězových e-mailů

Další informací, které se budeme věnovat jsou, `Štítky`. Pojďme se podívat, jak se vyvíjel seznam nejprobíranějších témat v čase. 

In [ ]:
N = 10
period = "14D" # 14 calendar day frequency (see pandas Offset aliases for time series)

topics = df.set_index("Datum")["Štítky"].str.split("  ")
out = []
for week, data in topics[topics.index.year > 2019].groupby(pd.Grouper(freq=period)):
    week_words = reduce(
        lambda a, b: (a if isinstance(a, list) else [])
        + (b if isinstance(b, list) else []),
        data.values.tolist(),
    )
    counter = Counter(preprocess(week_words))
    data = {}
    for i, w in enumerate(counter.most_common()[:N]):
        data["rank"] = i
        data["type"] = [w[0]]
        data["count"] = [w[1]]
        out.append(pd.DataFrame(data, index=[week]))

In [ ]:
trends = pd.concat(out)

In [ ]:
fig = (
    trends.groupby("type")
    .filter(lambda x: x.shape[0] > 4)
    .reset_index()
    .plot(
        backend="plotly",
        kind="bar",
        x="index",
        y="count",
        color="type",
        text="type",
        title="Graf vývoje témat v čase",
        color_discrete_sequence=px.colors.qualitative.Dark24,
    )
)
fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(barmode="stack")

> *🔦 Cvičení*: Dokážete najít nějaká témata, které spolu nejspíše souvisejí?

In [ ]:
show_events()

🔎 Ještě se můžete podívat na to, jak vývoj v probíraných tématech může souviset s událostmi.

# 📰 A co zdroje?

Pro dobrou argumentaci je třeba podložit svá tvrzení zdroji. I toto se děje ve světě řetězových e-mailů. Pojďme se společně podívat na to, jaké zdroje jsou nejcitovanější. 

In [ ]:
source, num_references = zip(*Counter(filter(lambda x: x, meta.Zdroj)).items())

In [ ]:
sources = pd.DataFrame.from_dict(dict(source=source, references=num_references))

In [ ]:
k = 15 # vezmeme top 15
sources.sort_values(by="references", ascending=False).iloc[:k].plot(
    kind="bar",
    x="source",
    y="references",
    backend="plotly",
    title=f"Top {k} zdrojů zmíněných v řetezových e-mailech",
)

🔎 Většina z těchto zdojů jsou "staří známí" na [dezinformační scéně](https://cs.wikipedia.org/wiki/Seznam_dezinforma%C4%8Dn%C3%ADch_web%C5%AF_v_%C4%8De%C5%A1tin%C4%9B), dále jsou tu specifikované i nespecifikované profily na sociálních sítích.

In [ ]:
cloud = wordcloud.WordCloud(max_font_size=70, width=1000, height=500)
cloud.fit_words(Counter(meta.Zdroj))
plt.figure(figsize=(16, 10)) # další wordcloud ;)
plt.imshow(cloud) #interpolation="bicubic"
plt.axis("off")
plt.show()

# 🗝️ Extrakce klíčových slov

Algoritmů pro detekci klíčových slov existuje několik. Ukážeme si, jaké výsledky nám vrátí algoritmus [RAKE (RApid Keyword Extractor)](https://www.analyticsvidhya.com/blog/2021/10/rapid-keyword-extraction-rake-algorithm-in-natural-language-processing/).

In [ ]:
from multi_rake import Rake, stopwords as sw

In [ ]:
rake = Rake(language_code="cs", stopwords=stop_words, max_words=2, max_words_unknown_lang=1, min_freq=15)

In [ ]:
text = ' '.join(meta.assign(len=meta["Tělo"].apply(len)).sort_values(by="len", ascending=False).iloc[:100]["Tělo"].values)

In [ ]:
print(text[:250])

In [ ]:
keywords = rake.apply(text)

In [ ]:
keywords[:15]

🔎 Můžeme si všimnout, že některá klíčová slova jsme detekovali stejně, některé jsou úplně jiná. Bylo by třeba použít nějaký postprocessing.

# 🧲  Souběžný výskyt štítků a zmínek

Dále nás může zajímat, jaké dvojice mezi zmínkami a štítky se často opakují.

In [ ]:
from tqdm.notebook import tqdm
import networkx as nx
import plotly.graph_objects as go

In [ ]:
coocurences = (
    df[["Štítky", "Zmínění"]]
    .fillna("")
    .apply(
        lambda row: [
            tuple(sorted([x, y]))
            for x in row["Štítky"].split("  ")
            for y in row["Zmínění"].split("  ")
        ],
        axis=1,
    )
)

In [ ]:
coocurences = reduce(operator.add, tqdm(coocurences.values))

In [ ]:
ctr = Counter(coocurences)

In [ ]:
G = nx.Graph()
for edge, w in ctr.most_common()[:40]:
    if edge[0] != edge[1]:
        G.add_edge(*edge, weight=w, title=w)

In [ ]:
from IPython.display import HTML

In [ ]:
from pyvis.network import Network

coocurences = Network(notebook=True)
coocurences.from_nx(G, edge_scaling=True, default_edge_weight=1)
coocurences.toggle_hide_edges_on_drag(True)
display(HTML(f"<h4>Graf souběžných výskytů zmínek a témat</h4>"))
coocurences.show("coocurences.html")

🔎 Nejsilnější souběžný výskyt je mezi Ruskem a USA, společně s nimi se řešilo hodně téma Ukrajiny. Dále se řešila česká politika a s ní související subjekty, nebo Evropská unie.

# ✳️ Vizualizace jednotlivých shluků řetězových mailů

Jako předposlední se podíváme na shluky e-mailů se stejnými vzory. Pomocí `networkx` vytvoříme graf, ve kterém vrcholy budou jednotlivé e-maily (modře 🔵) a jejich zdroje (oranžově 🟠). Jednotlivé e-maily budou také propojené se sobě podobnými.

In [ ]:
G = nx.Graph()

In [ ]:
meta["Datum"] = pd.to_datetime(meta["Datum"])

In [ ]:
df["Štítky"].fillna("", inplace=True)

👉 Mezi e-maily budeme měřit podobnost mezi štítky pomocí `Jaccardovy vzdálenosti`.

In [ ]:
def jaccard(u, v, zero_division=1):
    if not u and not v:
        return zero_division

    return len(u & v) / len(u | v)

In [ ]:
jaccard(set(['a', 'h', 'o', 'j']), set(['h', 'o', 'j']))

👉 Mezi dny budeme také měřit, jak dlouhý je časový odstup mezi dvojicí podobných e-mailů (opět ve dnech).

In [ ]:
def get_commons(u, v):
    """Populates the graph. """
    global G
    payload = {
        "type_": "similar",
        "color": "blue",
        "weight": 0.5,
        "label": None,
        "title": np.nan,
    }

    G.add_node(u, title=f"{u}\n", color="blue", type_="mail")
    G.add_node(v, title=f"{v}\n", color="blue", type_="mail")

    try:
        U = meta.loc[u]
        V = meta.loc[v]

        G.nodes[u]['title'] = G.nodes[u]['title'] + U['Předmět']
        G.nodes[v]['title'] = G.nodes[u]['title'] + V['Předmět']
        
        if U["Zdroj"]:
            G.add_node(U["Zdroj"], title=U["Zdroj"], type_="source", color="orange")
            G.add_edge(u, U["Zdroj"], color="orange")

        if V["Zdroj"]:
            G.add_node(V["Zdroj"], title=V["Zdroj"], type_="source", color="orange")
            G.add_edge(v, V["Zdroj"], type_="source", color="orange")

        payload["label"] = payload["title"] = abs((U.Datum - V.Datum).days)

        tmp = df[(df.ID == u)]["Štítky"]
        tags_u = set(tmp.iloc[0].split("  ") if tmp.shape[0] else "")

        tmp = df[(df.ID == v)]["Štítky"]
        tags_v = set(tmp.iloc[0].split("  ") if tmp.shape[0] else "")

        payload["weight"] = payload["sim"] = jaccard(tags_u, tags_v, zero_division=1)
    except KeyError:
        pass
    return payload

👉 Atributy hran pak vypadají například takto:

In [ ]:
get_commons(4860, 4309)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def add_to_graph(series):
    global G
    for key, value in tqdm(series.items(), total=series.shape[0]):
        for dest in value:
            commons = get_commons(key, dest)
            G.add_edge(key, dest, **commons)

In [ ]:
meta["similar"].str.split(",").apply(
    lambda x: list(map(int, filter(bool, x))) if x else []
).pipe(add_to_graph)

In [ ]:
components = list(nx.connected_components(G))

In [ ]:
components_sizes = np.fromiter(map(len, components), dtype=np.int32)

In [ ]:
print(f"👉 Průměrná velikost shluku je {components_sizes.mean():.3f} vrcholů se směrodatnou odchylkou {components_sizes.std():.3f}")

In [ ]:
components = sorted(
    filter(lambda x: len(x) > 5, list(nx.connected_components(G))),
    key=lambda x: len(x),
    reverse=True,
)

In [ ]:
components_sizes = components_sizes[1:]

In [ ]:
df["Zmínění"].fillna("", inplace=True)

In [ ]:
def show_cluster(index):
    S = G.subgraph(components[index])

    ctr = Counter(
        filter(
            lambda x: x,
            reduce(
                operator.add,
                [
                    df[df.ID == x]["Štítky"].str.split("  ").values[0]
                    for x, attr in S.nodes(data=True)
                    if attr["type_"] != "source" and x in df.ID.unique()
                ],
            )
            + reduce(
                operator.add,
                [
                    df[df.ID == x]["Zmínění"].str.split("  ").values[0]
                    for x, attr in S.nodes(data=True)
                    if attr["type_"] != "source" and x in df.ID.unique()
                ],
            ),
        )
    )

    nt = Network(
        notebook=True,
    )
    nt.from_nx(S, edge_scaling=False)
    nt.toggle_hide_edges_on_drag(True)
    nt.toggle_physics(False)
    nt.toggle_stabilization(False)
    display(HTML(f"<h1>Graf řetězových mailů na téma: {ctr.most_common()[0][0]}</h1>"))
    display(nt.show("nx.html"))
    
widgets.interact(show_cluster, index=range(1, len(components)));

🔎 Díky interaktivnímu grafu můžete shluk prozkoumat zblízka.

## 🪞 Jak to vypadá s podobností štítků mezi podobnými e-maily?

In [ ]:
tag_sim = np.array([attr["width"] for u,v, attr in G.edges(data=True) if 'width' in attr])
plt.hist(tag_sim)
plt.title("Histogram Jaccardovy podobnosti mezi štítky jednotlivých e-mailů")
plt.show()

🔎 Z grafu je vidět, že ve většině případů jsou štítky úplně stejné nebo se jen lehce liší. Pak jsou tu e-maily, které mají menší podobnost - těch je ale méně.

## ⌚ Jak dlouho žije takový řetězový mail?

In [ ]:
days_diff = np.array(
    [a["title"] for src, dest, a in G.edges(data=True) if "title" in a]
)
plt.hist(days_diff, density=False, bins=52, log=True)
plt.title("Histogram počtu dnů mezi zaregistrováním podobného e-mailu (logaritmická škála) ")
plt.show()

🔎 Z histogramu vidíme, že většina e-mailů se šíří pouze krátkodobě, ale některé e-maily dokáží přežít až skoro 3 roky.

# 🕸️ Phrase net a POS-tagging

Jako poslední si uděláme vlastní "phrase net" a navíc obarvíme vrcholy podle toho, jaký [POS tag](https://en.wikipedia.org/wiki/Part-of-speech_tagging) má dané slovo (tj. proces označování slov v textu (korpusu)  odpovídajících určité části řeči na základě jejich definice a kontextu jako např. podstatná jména, slovesa, citoslovce apod.). 

👉 Pojďme se podívat na to, jaké POS tagy má text z e-mailu s nejdelším tělem.

In [ ]:
import unidecode

In [ ]:
words = (
    unidecode.unidecode(
        meta.assign(len=meta["Tělo"].apply(len))
        .sort_values(by="len", ascending=True)["Tělo"]
        .iloc[-1]
    )
    .replace("\r\n", "").replace(","," ")
    .split()
)
print(words[:15])

In [ ]:
def get_pos_tag(word):
    """Returns a POS tag for given word."""
    m = morph.find(word)
    pos = None
    try:
        pos = m[0]["tags"]["pos"]
    except (KeyError, IndexError):
        pass
    return pos

In [ ]:
list(filter(lambda x: x[1] is not None, zip(words, map(get_pos_tag, words))))[:15]

👉 Vizualizovat takto celý dataset by bylo výpočetně náročné. Zkusíme si to alespoň na úryvku textu o řetězových e-mailech z Wikipedie.

In [ ]:
#https://cs.wikipedia.org/wiki/%C5%98et%C4%9Bzov%C3%BD_e-mail
sentences = "Řetězový e-mail je hromadně přeposílaná zpráva, která se exponenciálně šíří prostřednictvím soukromé internetové komunikace. Jejím obsahem jsou často dezinformace, hoaxy a propaganda, provázanost s konkrétní osobou mu dodává mezi čtenáři validitu. Původně se jednalo o obdobu řetězových dopisů, od nich se dnešní (2019) texty řetězových e-mailů odlišují zejména absencí povinnosti přeposlání zprávy spojené se slibem profitu v případě přeposlání a výhrůžky, pokud k přeposlání nedojde. Nejvíce jsou řetězovými e-maily zasaženi uživatelé internetu starší 65 let"
print(sentences)
sentences = re.sub("[,\.]", " ", sentences).split(".")

In [ ]:
SG = nx.DiGraph()

In [ ]:
colors = (x for x in px.colors.qualitative.Dark24)
mapping = {}

for sentence in sentences:
    tokens = re.sub("[^\w0-9\s]+", "", sentence).split(" ")
    for i, node in enumerate(tokens):
        pos = get_pos_tag(node)
        node = lemmatize(node)
        if pos not in mapping:
            mapping[pos] = next(colors)
        SG.add_node(node, pos=pos, color=mapping[pos], title=pos or "unknown")
        if i > 0:
            SG.add_edge(lemmatize(tokens[i - 1]), node)

In [ ]:
nt = Network(notebook=True, directed=True,   neighborhood_highlight=True)
nt.from_nx(SG)

nt.toggle_hide_edges_on_drag(True)
nt.toggle_stabilization(True)
display(HTML(f"<h1>🏷️ Vizualizace POS tagů pro text o řetězových e-mailech</h1>"))
nt.show("nx2.html")